In [2]:
%run ./read_file.ipynb 

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6a979f90-d9fb-4ae0-b538-fa00598ae025;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 195ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/05 17:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 17:01:45 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/05 17:01:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_labels = readCheckpoint('labels')
#create a timestamp column and drop the old timestamp - does not appear necessary
df_labels = df_labels.withColumn('ts2', to_timestamp(col('timestamp'))).drop('timestamp')
#check new col data types
df_labels.printSchema()

 5:01PM UTC on Mar 05, 2023 --- read and cache time: 7.392996311187744 seconds ---
root
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [ ]:
def build_adjaceny(df, dataset = 'ecar'):
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    #create ref dict
    dict_node_edge_events = {}
    
    #for all actors in actIDs
    for aID in actIDs: 
        first_id = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()[0]['id']
        
        #now add this to the dictionary as a key
        first_event = str(aID+':'+first_id)
        dict_node_edge_events[first_event] = []
        
        #now add all the events where the actorID is present but not the objectID, group by unique object ID
        #and pull out the first event - and I checked and believe the sorting holds through grouping
        col_select_children_events = df.select('objectID','id')\
        .where((col("actorID") == aID) & \
                 (col("objectID") != aID)).groupBy("objectID").agg(first("id")).collect() 
        
        #add children objects into dictioinary 
        dict_node_edge_events[first_event] = [str(row["objectID"]+':'+row["first(id)"]) for row in col_select_children_events]
        
    return dict_node_edge_events